In [4]:
import os
import logging
from obspy import read, Stream, UTCDateTime
import numpy as np
from obspy import Trace

station = 'CPO'
year = '2023'
channels = ['BHN','BHE','BHZ']

for channel in channels:
    file_path = f'/home/gridsan/mknuth/01_Seismic_Wave_Data_Prediction/01_Data/01_Seismic_Wave_Data/{year}/{station}/{channel}/{station}_{channel}_{year}-01-01.mseed'
    stream = read(file_path)
    print(stream)

1 Trace(s) in Stream:
CI.CPO..BHN | 2023-01-01T00:00:00.024999Z - 2023-01-01T23:59:59.974999Z | 40.0 Hz, 3455999 samples
1 Trace(s) in Stream:
CI.CPO..BHE | 2023-01-01T00:00:00.000000Z - 2023-01-01T23:59:59.975000Z | 40.0 Hz, 3456000 samples
1 Trace(s) in Stream:
CI.CPO..BHZ | 2023-01-01T00:00:00.024999Z - 2023-01-01T23:59:59.974999Z | 40.0 Hz, 3455999 samples


In [5]:
stream[0].stats

         network: CI
         station: CPO
        location: 
         channel: BHZ
       starttime: 2023-01-01T00:00:00.024999Z
         endtime: 2023-01-01T23:59:59.974999Z
   sampling_rate: 40.0
           delta: 0.025
            npts: 3455999
           calib: 1.0
         _format: MSEED
           mseed: AttribDict({'dataquality': 'D', 'number_of_records': 8824, 'encoding': 'STEIM2', 'byteorder': '>', 'record_length': 512, 'filesize': 4517888})

In [6]:
import os
import logging
from obspy import read, Stream, UTCDateTime
import numpy as np
from obspy import Trace

def format_duration(duration):
    """Convert duration from seconds to hours:minutes format."""
    hours = int(duration // 3600)
    minutes = int((duration % 3600) // 60)
    return f"{hours:02d}:{minutes:02d}"

def save_processed_trace(trace, original_path):
    """Save the processed trace to a new file with '_processed' appended to the original name."""
    directory, filename = os.path.split(original_path)
    name, ext = os.path.splitext(filename)
    new_filename = f"{name}_processed{ext}"
    new_path = os.path.join(directory, new_filename)
    trace.write(new_path, format='MSEED')
    logging.info(f"Saved processed trace to: {new_path}")

def process_waveform(file_path):
    try:
        stream = read(file_path)
        #print(f"Number of traces: {len(stream)}")
        if len(stream) > 1:
            print(stream)
        
        
        stream.merge(method=1, fill_value=0)
        duration = stream[0].stats.endtime - stream[0].stats.starttime
        duration_str = format_duration(duration)
        logging.info(f"Stream goes from {stream[0].stats.starttime} to {stream[0].stats.endtime} (duration: {duration_str})")
        #print(f"Stream goes from {stream[0].stats.starttime} to {stream[0].stats.endtime} (duration: {duration_str})")

        # Save the processed trace
        save_processed_trace(stream[0], file_path)

    except Exception as e:
        logging.error(f"Error processing file {stream[0]}: {str(e)}")
        print(f"Error processing file {stream[0]}: {str(e)}")



base_dir = f"/home/gridsan/mknuth/01_Seismic_Wave_Data_Prediction/01_Data/01_Seismic_Wave_Data/{year}/{station}"

logging.basicConfig(filename=f"{year}_{station}_trace_preprocessing.log", level=logging.INFO, format='%(asctime)s - %(message)s', filemode='w')
# Base directory to start searching


print(f'processing year {year}')
# Collect all mseed files recursively in a list
file_list = []

for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".mseed"):
            file_path = os.path.join(root, file)
            file_list.append(file_path)

# Sort the list of files alphabetically
file_list.sort()
print(f"In total {len(file_list)} files")

# Process each file that has not been processed yet, total 1095 in 2021
for file_path in file_list:
    # Skip already processed files
    if "_processed" in os.path.basename(file_path):
        logging.info(f"Skipping already processed file: {os.path.basename(file_path)}")
        continue
    
    #print(f"Processing file {file_path}")
    logging.info(f"Processing file {os.path.basename(file_path)}")
    process_waveform(file_path)

processing year 2023
In total 2190 files
4 Trace(s) in Stream:
CI.CPO..BHE | 2023-01-03T00:00:00.000000Z - 2023-01-03T13:24:08.750000Z | 40.0 Hz, 1929951 samples
CI.CPO..BHE | 2023-01-03T13:24:20.275000Z - 2023-01-03T13:24:44.250000Z | 40.0 Hz, 960 samples
CI.CPO..BHE | 2023-01-03T13:24:56.175000Z - 2023-01-03T13:25:20.325000Z | 40.0 Hz, 967 samples
CI.CPO..BHE | 2023-01-03T13:28:48.625000Z - 2023-01-03T23:59:59.975000Z | 40.0 Hz, 1514855 samples
5 Trace(s) in Stream:
CI.CPO..BHE | 2023-01-04T00:00:00.000000Z - 2023-01-04T16:14:51.800000Z | 40.0 Hz, 2339673 samples
CI.CPO..BHE | 2023-01-04T16:15:00.975000Z - 2023-01-04T16:16:23.100000Z | 40.0 Hz, 3286 samples
CI.CPO..BHE | 2023-01-04T16:17:04.450000Z - 2023-01-04T16:17:29.925000Z | 40.0 Hz, 1020 samples
CI.CPO..BHE | 2023-01-04T16:17:46.825000Z - 2023-01-04T16:18:08.125000Z | 40.0 Hz, 853 samples
CI.CPO..BHE | 2023-01-04T16:18:22.525000Z - 2023-01-04T23:59:59.975000Z | 40.0 Hz, 1107899 samples
2 Trace(s) in Stream:
CI.CPO..BHE | 2023-0

In [ ]:
file_path = '01_Data/01_Seismic_Wave_Data/2024/SDD/BHZ/SDD_BHZ_2024-11-21.mseed'

stream = read(file_path)
stream.merge()
stream